In [112]:
import numpy as np
import pandas as pd
import sys, copy, os, shutil
from tqdm.notebook import tqdm

In [154]:
# create a dataframe for logs of which runs have finished successfully or not
logs = pd.DataFrame(data=None, columns=["dataset", "variant", "seed", "finished_epochs"])

# go thru all resnet variants and see which ones succeeded and/or failed
for dataset in ["MNIST", "FashionMNIST", "CIFAR10"]:
    for variant in [20, 30, 44]:
        for seed in range(50):
            
            # assemble our foldername + check if it exists
            foldername = f"models/{dataset}/resnet_variant={variant}_seed={str(seed).zfill(3)}"
            if os.path.exists(foldername) == False:
                finished_epochs = -1
            elif len(os.listdir(foldername)) == 0:
                finished_epochs = 0
            else:
                finished_epochs = len([f for f in os.listdir(foldername) if ".pth" in f])
                
            # create our row
            logs.loc[len(logs.index)] = [dataset, variant, seed, finished_epochs]

In [122]:
# creating new payload for CIFAR-10 RESNET re-runs
cifar_fails = copy.deepcopy(logs.query("finished_epochs != 100 and dataset == 'CIFAR10'")).reset_index(drop=True)
cifar_fails["var_id"] = [{20 : 0, 30 : 1, 44 : 2}[val] for val in cifar_fails.variant]

In [155]:
# creating new payload for MNIST family RESNET re-runs
mnist_fails = copy.deepcopy(logs.query("finished_epochs != 100 and dataset != 'CIFAR10'")).reset_index(drop=True)
mnist_fails["var_id"] = [{20 : 0, 30 : 1, 44 : 2}[val] for val in mnist_fails.variant]

In [156]:
mnist_fails

,dataset,variant,seed,finished_epochs,var_id
0,MNIST,30,0,-1,1
1,MNIST,30,1,-1,1
2,MNIST,30,2,-1,1
3,MNIST,30,3,-1,1
4,MNIST,30,4,-1,1
...,...,...,...,...,...
172,FashionMNIST,44,27,0,2
173,FashionMNIST,44,28,0,2
174,FashionMNIST,44,29,0,2
175,FashionMNIST,44,31,0,2


In [160]:
mnist_fails.to_csv("mnist_fails.csv", index=False)

In [170]:
dataset, variant, seed = pd.read_csv("mnist_fails.csv").loc[0][["dataset", "var_id", "seed"]].values

In [175]:
mnist_fails

,dataset,variant,seed,finished_epochs,var_id
0,MNIST,30,0,-1,1
1,MNIST,30,1,-1,1
2,MNIST,30,2,-1,1
3,MNIST,30,3,-1,1
4,MNIST,30,4,-1,1
...,...,...,...,...,...
172,FashionMNIST,44,27,0,2
173,FashionMNIST,44,28,0,2
174,FashionMNIST,44,29,0,2
175,FashionMNIST,44,31,0,2


In [174]:
type(seed)

numpy.int64

In [158]:
cifar_fails

,dataset,variant,seed,finished_epochs,var_id
0,CIFAR10,20,39,14,0
1,CIFAR10,30,0,-1,1
2,CIFAR10,30,1,-1,1
3,CIFAR10,30,2,-1,1
4,CIFAR10,30,3,-1,1
...,...,...,...,...,...
60,CIFAR10,44,41,0,2
61,CIFAR10,44,42,0,2
62,CIFAR10,44,45,0,2
63,CIFAR10,44,47,0,2


In [159]:
# save our CIFAR-10 fails so that we can rerun (65 of them!)
cifar_fails.to_csv("cifar_fails.csv", index=False)